## 1. Libraries and Setup

In [1]:
from google.colab import drive
import json
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp /content/gdrive/MyDrive/converted_224x224.tar.gz /content
%cd /content
!tar -xzf converted_224x224.tar.gz

/content


In [3]:
import os
import sys
import time

import numpy as np
import pandas as pd

import pdb
import gc
from tqdm.notebook import trange, tqdm
from PIL import Image

import torch
import torchvision 
from torchvision import transforms
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
# from torchinfo import summary

import warnings
warnings.filterwarnings('ignore')


In [4]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

num_workers = 8 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  False  with num_workers =  0  system version =  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


## 2. Data Loading

### 2.1 Load Data

In [5]:
np.random.seed(0)
df = pd.read_csv("4+1.csv")
df = df.iloc[:, 1:]

train_end = int(len(df)*0.7)
val_end = int(len(df)*0.85)
train_data = df[:train_end]
val_data = df[train_end:val_end]
test_data = df[val_end:]

In [6]:
train_data = train_data.reset_index().drop('Unnamed: 0.1',1).drop('index',1)
val_data = val_data.reset_index().drop('Unnamed: 0.1',1).drop('index',1)
test_data = test_data.reset_index().drop('Unnamed: 0.1',1).drop('index',1)

In [7]:
test_data

,frames,label
0,"('frame_149670.jpg', 'frame_149739.jpg', 'fram...",16
1,"('frame_149738.jpg', 'frame_149692.jpg', 'fram...",80
2,"('frame_149729.jpg', 'frame_149716.jpg', 'fram...",86
3,"('frame_149669.jpg', 'frame_149712.jpg', 'fram...",7
4,"('frame_149683.jpg', 'frame_149707.jpg', 'fram...",34
...,...,...
17942,"('frame_180809.jpg', 'frame_174579.jpg', 'fram...",101
17943,"('frame_174581.jpg', 'frame_174634.jpg', 'fram...",14
17944,"('frame_174605.jpg', 'frame_174596.jpg', 'fram...",54
17945,"('frame_181123.jpg', 'frame_174585.jpg', 'fram...",96


### 2.2 Custom Dataset Class

In [8]:
# cur_dir = "data/qscale31_unique/"
cur_dir = "/content/converted_224x224/"

In [9]:
# Define dataset class
class MyDataSet(Dataset):

    # load the dataset
    def __init__(self, data, transform=None, **kwargs):
        self.X = data["frames"]
        self.Y = data["label"]
        self.transform = transform

    # get number of items/rows in dataset
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        for c in ["(",")",",","'"]:
            x = x.replace(c, "")
        x = x.split(" ")
        images = []
        for img_file in x:
            img = Image.open(cur_dir + img_file)
            img = torchvision.transforms.ToTensor()(img)
            # img = torchvision.transforms.Normalize(mean=[0.2068, 0.2242, 0.2269], std=[0.2520, 0.2394, 0.2320])(img)
            if self.transform:
                img = self.transform(img)
            images.extend(img)
        x = torch.stack(images)
        return x, y

In [10]:
train_set = MyDataSet(train_data)
train_set[0][0].shape

torch.Size([15, 224, 224])

### 2.3 Dataloader

In [11]:
# transform_train = torchvision.transforms.Compose([
#     torchvision.transforms.Normalize(mean=[0.4967, 0.4858, 0.4584, 0.4969, 0.4700, 0.4585, 0.4964, 0.4854, 0.4580, 0.4972, 0.4862, 0.4585, 0.4964, 0.4857, 0.4584],
#                                       std=[0.2681, 0.2574, 0.2433, 0.2678, 0.2500, 0.2429, 0.2679, 0.2572, 0.2428, 0.2680, 0.2573, 0.2428, 0.2680, 0.2574, 0.2431])
# ])

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0.15)
])

In [12]:
batch_size = 128

# training data
train_set = MyDataSet(train_data, transform_train)
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, num_workers=8)

# validation data
val_set = MyDataSet(val_data)
val_loader = DataLoader(val_set, shuffle=False, batch_size=batch_size, num_workers=8)

# test data
test_set = MyDataSet(test_data)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size, num_workers=8)

In [13]:
# def batch_mean_and_sd(loader, num_channels):
    
#     cnt = 0
#     fst_moment = torch.empty(num_channels)
#     snd_moment = torch.empty(num_channels)

#     for images, _ in tqdm(loader):
#         b, c, h, w = images.shape
#         nb_pixels = b * h * w
#         sum_ = torch.sum(images, dim=[0, 2, 3])
#         sum_of_square = torch.sum(images ** 2,
#                                   dim=[0, 2, 3])
#         fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
#         snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)
#         cnt += nb_pixels

#     mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)        
#     return mean,std
  
# num_channels = 15
# mean, std = batch_mean_and_sd(train_loader, num_channels)
# print("mean and std: \n", mean, std)

# Model

In [14]:
# Code based on https://pytorch.org/vision/stable/_modules/torchvision/models/regnet.html

import math
import torch

from collections import OrderedDict
from functools import partial
from typing import Any, Callable, List, Optional, Tuple
from torch import nn, Tensor

__all__ = ["RegNet", "regnet_y_400mf", "regnet_y_800mf", "regnet_y_1_6gf",
           "regnet_y_3_2gf", "regnet_y_8gf", "regnet_y_16gf", "regnet_y_32gf",
           "regnet_x_400mf", "regnet_x_800mf", "regnet_x_1_6gf", "regnet_x_3_2gf",
           "regnet_x_8gf", "regnet_x_16gf", "regnet_x_32gf"]

class ConvNormActivation(torch.nn.Sequential):
    """
    Configurable block used for Convolution-Normalzation-Activation blocks.
    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the Convolution-Normalzation-Activation block
        kernel_size: (int, optional): Size of the convolving kernel. Default: 3
        stride (int, optional): Stride of the convolution. Default: 1
        padding (int, tuple or str, optional): Padding added to all four sides of the input. Default: None, in wich case it will calculated as ``padding = (kernel_size - 1) // 2 * dilation``
        groups (int, optional): Number of blocked connections from input channels to output channels. Default: 1
        norm_layer (Callable[..., torch.nn.Module], optional): Norm layer that will be stacked on top of the convolutiuon layer. If ``None`` this layer wont be used. Default: ``torch.nn.BatchNorm2d``
        activation_layer (Callable[..., torch.nn.Module], optinal): Activation function which will be stacked on top of the normalization layer (if not None), otherwise on top of the conv layer. If ``None`` this layer wont be used. Default: ``torch.nn.ReLU``
        dilation (int): Spacing between kernel elements. Default: 1
        inplace (bool): Parameter for the activation layer, which can optionally do the operation in-place. Default ``True``
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        stride: int = 1,
        padding: Optional[int] = None,
        groups: int = 1,
        norm_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.BatchNorm2d,
        activation_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.ReLU,
        dilation: int = 1,
        inplace: bool = True,
    ) -> None:
        if padding is None:
            padding = (kernel_size - 1) // 2 * dilation
        layers = [
            torch.nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                dilation=dilation,
                groups=groups,
                bias=norm_layer is None,
            )
        ]
        if norm_layer is not None:
            layers.append(norm_layer(out_channels))
        if activation_layer is not None:
            layers.append(activation_layer(inplace=inplace))
        super().__init__(*layers)
        # _log_api_usage_once(self)
        self.out_channels = out_channels


class SqueezeExcitation(torch.nn.Module):
    """
    This block implements the Squeeze-and-Excitation block from https://arxiv.org/abs/1709.01507 (see Fig. 1).
    Parameters ``activation``, and ``scale_activation`` correspond to ``delta`` and ``sigma`` in in eq. 3.
    Args:
        input_channels (int): Number of channels in the input image
        squeeze_channels (int): Number of squeeze channels
        activation (Callable[..., torch.nn.Module], optional): ``delta`` activation. Default: ``torch.nn.ReLU``
        scale_activation (Callable[..., torch.nn.Module]): ``sigma`` activation. Default: ``torch.nn.Sigmoid``
    """

    def __init__(
        self,
        input_channels: int,
        squeeze_channels: int,
        activation: Callable[..., torch.nn.Module] = torch.nn.ReLU,
        scale_activation: Callable[..., torch.nn.Module] = torch.nn.Sigmoid,
    ) -> None:
        super().__init__()
        # _log_api_usage_once(self)
        self.avgpool = torch.nn.AdaptiveAvgPool2d(1)
        self.fc1 = torch.nn.Conv2d(input_channels, squeeze_channels, 1)
        self.fc2 = torch.nn.Conv2d(squeeze_channels, input_channels, 1)
        self.activation = activation()
        self.scale_activation = scale_activation()

    def _scale(self, input: Tensor) -> Tensor:
        scale = self.avgpool(input)
        scale = self.fc1(scale)
        scale = self.activation(scale)
        scale = self.fc2(scale)
        return self.scale_activation(scale)

    def forward(self, input: Tensor) -> Tensor:
        scale = self._scale(input)
        return scale * input

def _make_divisible(v: float, divisor: int, min_value: Optional[int] = None) -> int:
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


class SimpleStemIN(ConvNormActivation):
    """Simple stem for ImageNet: 3x3, BN, ReLU."""

    def __init__(
        self,
        width_in: int,
        width_out: int,
        norm_layer: Callable[..., nn.Module],
        activation_layer: Callable[..., nn.Module],
    ) -> None:
        super().__init__(width_in, width_out, kernel_size=3, stride=2,
                         norm_layer=norm_layer, activation_layer=activation_layer)


class BottleneckTransform(nn.Sequential):
    """Bottleneck transformation: 1x1, 3x3 [+SE], 1x1."""

    def __init__(
        self,
        width_in: int,
        width_out: int,
        stride: int,
        norm_layer: Callable[..., nn.Module],
        activation_layer: Callable[..., nn.Module],
        group_width: int,
        bottleneck_multiplier: float,
        se_ratio: Optional[float],
    ) -> None:
        layers: OrderedDict[str, nn.Module] = OrderedDict()
        w_b = int(round(width_out * bottleneck_multiplier))
        g = w_b // group_width

        layers["a"] = ConvNormActivation(width_in, w_b, kernel_size=1, stride=1,
                                         norm_layer=norm_layer, activation_layer=activation_layer)
        layers["b"] = ConvNormActivation(w_b, w_b, kernel_size=3, stride=stride, groups=g,
                                         norm_layer=norm_layer, activation_layer=activation_layer)

        if se_ratio:
            # The SE reduction ratio is defined with respect to the
            # beginning of the block
            width_se_out = int(round(se_ratio * width_in))
            layers["se"] = SqueezeExcitation(
                input_channels=w_b,
                squeeze_channels=width_se_out,
                activation=activation_layer,
            )

        layers["c"] = ConvNormActivation(w_b, width_out, kernel_size=1, stride=1,
                                         norm_layer=norm_layer, activation_layer=None)
        super().__init__(layers)


class ResBottleneckBlock(nn.Module):
    """Residual bottleneck block: x + F(x), F = bottleneck transform."""

    def __init__(
        self,
        width_in: int,
        width_out: int,
        stride: int,
        norm_layer: Callable[..., nn.Module],
        activation_layer: Callable[..., nn.Module],
        group_width: int = 1,
        bottleneck_multiplier: float = 1.0,
        se_ratio: Optional[float] = None,
    ) -> None:
        super().__init__()

        # Use skip connection with projection if shape changes
        self.proj = None
        should_proj = (width_in != width_out) or (stride != 1)
        if should_proj:
            self.proj = ConvNormActivation(width_in, width_out, kernel_size=1,
                                           stride=stride, norm_layer=norm_layer, activation_layer=None)
        self.f = BottleneckTransform(
            width_in,
            width_out,
            stride,
            norm_layer,
            activation_layer,
            group_width,
            bottleneck_multiplier,
            se_ratio,
        )
        self.activation = activation_layer(inplace=True)

    def forward(self, x: Tensor) -> Tensor:
        if self.proj is not None:
            x = self.proj(x) + self.f(x)
        else:
            x = x + self.f(x)
        return self.activation(x)


class AnyStage(nn.Sequential):
    """AnyNet stage (sequence of blocks w/ the same output shape)."""

    def __init__(
        self,
        width_in: int,
        width_out: int,
        stride: int,
        depth: int,
        block_constructor: Callable[..., nn.Module],
        norm_layer: Callable[..., nn.Module],
        activation_layer: Callable[..., nn.Module],
        group_width: int,
        bottleneck_multiplier: float,
        se_ratio: Optional[float] = None,
        stage_index: int = 0,
    ) -> None:
        super().__init__()

        for i in range(depth):
            block = block_constructor(
                width_in if i == 0 else width_out,
                width_out,
                stride if i == 0 else 1,
                norm_layer,
                activation_layer,
                group_width,
                bottleneck_multiplier,
                se_ratio,
            )

            self.add_module(f"block{stage_index}-{i}", block)


class BlockParams:
    def __init__(
        self,
        depths: List[int],
        widths: List[int],
        group_widths: List[int],
        bottleneck_multipliers: List[float],
        strides: List[int],
        se_ratio: Optional[float] = None,
    ) -> None:
        self.depths = depths
        self.widths = widths
        self.group_widths = group_widths
        self.bottleneck_multipliers = bottleneck_multipliers
        self.strides = strides
        self.se_ratio = se_ratio

    @classmethod
    def from_init_params(
        cls,
        depth: int,
        w_0: int,
        w_a: float,
        w_m: float,
        group_width: int,
        bottleneck_multiplier: float = 1.0,
        se_ratio: Optional[float] = None,
        **kwargs: Any,
    ) -> "BlockParams":
        """
        Programatically compute all the per-block settings,
        given the RegNet parameters.

        The first step is to compute the quantized linear block parameters,
        in log space. Key parameters are:
        - `w_a` is the width progression slope
        - `w_0` is the initial width
        - `w_m` is the width stepping in the log space

        In other terms
        `log(block_width) = log(w_0) + w_m * block_capacity`,
        with `bock_capacity` ramping up following the w_0 and w_a params.
        This block width is finally quantized to multiples of 8.

        The second step is to compute the parameters per stage,
        taking into account the skip connection and the final 1x1 convolutions.
        We use the fact that the output width is constant within a stage.
        """

        QUANT = 8
        STRIDE = 2

        if w_a < 0 or w_0 <= 0 or w_m <= 1 or w_0 % 8 != 0:
            raise ValueError("Invalid RegNet settings")
        # Compute the block widths. Each stage has one unique block width
        widths_cont = torch.arange(depth) * w_a + w_0
        block_capacity = torch.round(torch.log(widths_cont / w_0) / math.log(w_m))
        block_widths = (
            torch.round(torch.divide(w_0 * torch.pow(w_m, block_capacity), QUANT))
            * QUANT
        ).int().tolist()
        num_stages = len(set(block_widths))

        # Convert to per stage parameters
        split_helper = zip(
            block_widths + [0],
            [0] + block_widths,
            block_widths + [0],
            [0] + block_widths,
        )
        splits = [w != wp or r != rp for w, wp, r, rp in split_helper]

        stage_widths = [w for w, t in zip(block_widths, splits[:-1]) if t]
        stage_depths = torch.diff(torch.tensor([d for d, t in enumerate(splits) if t])).int().tolist()

        strides = [STRIDE] * num_stages
        bottleneck_multipliers = [bottleneck_multiplier] * num_stages
        group_widths = [group_width] * num_stages

        # Adjust the compatibility of stage widths and group widths
        stage_widths, group_widths = cls._adjust_widths_groups_compatibilty(
            stage_widths, bottleneck_multipliers, group_widths
        )

        return cls(
            depths=stage_depths,
            widths=stage_widths,
            group_widths=group_widths,
            bottleneck_multipliers=bottleneck_multipliers,
            strides=strides,
            se_ratio=se_ratio,
        )

    def _get_expanded_params(self):
        return zip(
            self.widths, self.strides, self.depths, self.group_widths, self.bottleneck_multipliers
        )

    @staticmethod
    def _adjust_widths_groups_compatibilty(
            stage_widths: List[int], bottleneck_ratios: List[float],
            group_widths: List[int]) -> Tuple[List[int], List[int]]:
        """
        Adjusts the compatibility of widths and groups,
        depending on the bottleneck ratio.
        """
        # Compute all widths for the current settings
        widths = [int(w * b) for w, b in zip(stage_widths, bottleneck_ratios)]
        group_widths_min = [min(g, w_bot) for g, w_bot in zip(group_widths, widths)]

        # Compute the adjusted widths so that stage and group widths fit
        ws_bot = [_make_divisible(w_bot, g) for w_bot, g in zip(widths, group_widths_min)]
        stage_widths = [int(w_bot / b) for w_bot, b in zip(ws_bot, bottleneck_ratios)]
        return stage_widths, group_widths_min


class RegNet(nn.Module):
    def __init__(
        self,
        block_params: BlockParams,
        in_channels: int = 3,
        num_classes: int = 1000,
        stem_width: int = 32,
        stem_type: Optional[Callable[..., nn.Module]] = None,
        block_type: Optional[Callable[..., nn.Module]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        activation: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()

        if stem_type is None:
            stem_type = SimpleStemIN
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if block_type is None:
            block_type = ResBottleneckBlock
        if activation is None:
            activation = nn.ReLU

        # Ad hoc stem
        self.stem = stem_type(
            in_channels,  # width_in
            stem_width,
            norm_layer,
            activation,
        )

        current_width = stem_width

        blocks = []
        for i, (
            width_out,
            stride,
            depth,
            group_width,
            bottleneck_multiplier,
        ) in enumerate(block_params._get_expanded_params()):
            blocks.append(
                (
                    f"block{i+1}",
                    AnyStage(
                        current_width,
                        width_out,
                        stride,
                        depth,
                        block_type,
                        norm_layer,
                        activation,
                        group_width,
                        bottleneck_multiplier,
                        block_params.se_ratio,
                        stage_index=i + 1,
                    ),
                )
            )

            current_width = width_out

        self.trunk_output = nn.Sequential(OrderedDict(blocks))

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_features=current_width, out_features=num_classes)

        # Init weights and good to go
        self._reset_parameters()

        # initial weights
        for m in self.modules():
          if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode="fan_out")
            if m.bias is not None:
                nn.init.zeros_(m.bias)
          elif isinstance(m, nn.BatchNorm2d):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)
          elif isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.zeros_(m.bias)

    def forward(self, x: Tensor) -> Tensor:
        x = self.stem(x)
        x = self.trunk_output(x)

        x = self.avgpool(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)

        return x

    def _reset_parameters(self) -> None:
        # Performs ResNet-style weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Note that there is no bias due to BN
                fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                nn.init.normal_(m.weight, mean=0.0, std=math.sqrt(2.0 / fan_out))
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0.0, std=0.01)
                nn.init.zeros_(m.bias)


def _regnet(arch: str, block_params: BlockParams, in_channels, num_classes, pretrained: bool, progress: bool, **kwargs: Any) -> RegNet:
    model = RegNet(block_params, in_channels=in_channels, num_classes=num_classes, norm_layer=partial(nn.BatchNorm2d, eps=1e-05, momentum=0.1), **kwargs)
    if pretrained:
        if arch not in model_urls:
            raise ValueError(f"No checkpoint is available for model type {arch}")
        # state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        # model.load_state_dict(state_dict)
    return model

def regnet_y_400mf(in_channels, num_classes, pretrained: bool = False, progress: bool = True, **kwargs: Any) -> RegNet:
    """
    Constructs a RegNetY_400MF architecture from
    `"Designing Network Design Spaces" <https://arxiv.org/abs/2003.13678>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    params = BlockParams.from_init_params(depth=16, w_0=48, w_a=27.89, w_m=2.09,
                                          group_width=8, se_ratio=0.25, **kwargs)
    return _regnet("regnet_y_400mf", params, in_channels, num_classes, pretrained, progress, **kwargs)

In [15]:
class ClassificationNetwork(torch.nn.Module):
    def __init__(self, in_channels, embedding, num_classes):
        super().__init__()
        self.cnn = regnet_y_400mf(in_channels, embedding)
        self.mlp = nn.Sequential(nn.Linear(embedding,num_classes))


    def forward(self, x):
      out = self.cnn(x)
      out = self.mlp(out)
      return out

In [17]:
numEpochs = 100
in_features = 15 # TODO: change RGB channels according to num of frames
embedding = 512

learningRate = 0.1
weightDecay = 1e-4

num_classes = 120

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = ClassificationNetwork(in_features, embedding, num_classes)
network = network.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)

In [ ]:
# Train!
max_val_acc = 0
for epoch in range(11, numEpochs):
    # Train
    network.train()
    avg_loss = 0.0
    avg_train_acc = 0.0
    for batch_num, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        x.requires_grad_()
        x, y = x.to(device), y.to(device)
        outputs = network(x)
        num_train_correct = (torch.argmax(outputs, axis=1) == y).sum().item()
        num_labels = len(y)
        avg_train_acc += (num_train_correct/num_labels)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()
        avg_loss += loss.item()

        if batch_num % 50 == 49:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}\tTraining Accuracy : {:.4f}'.format(epoch, batch_num+1, avg_loss/50, avg_train_acc/50))
            avg_loss = 0.0
            avg_train_acc = 0.0

        torch.cuda.empty_cache()
        del x
        del y
        del loss
    
    # Validate
    with torch.no_grad():
      network.eval()
      avg_val_loss = 0.0
      num_correct = 0
      for batch_num, (x, y) in enumerate(val_loader):
          x, y = x.to(device), y.to(device)
          outputs = network(x)
          num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
          loss = criterion(outputs, y.long())
          avg_val_loss += loss.item()

          torch.cuda.empty_cache()
          del x
          del y

      avg_val_loss = avg_val_loss / len(val_loader)
      val_acc = num_correct / len(val_set)
      # checkpoint_name = "/content/gdrive/MyDrive/dl_project/model_checkpoints/efficientnet2/model_" + str(epoch) + ".pt"
      # torch.save(network.state_dict(), checkpoint_name)
      if val_acc > max_val_acc:
          max_val_acc = val_acc
          torch.save(network.state_dict(), "/content/gdrive/MyDrive/regnet_ours_data4/best_model_" + str(epoch) + ".pt")
    scheduler.step(avg_val_loss)
    print('Epoch: {}, Validation Loss: {:.3f}, Validation Accuracy: {:.3f}'.format(epoch, avg_val_loss, val_acc))

Epoch: 11	Batch: 50	Avg-Loss: 2.0942	Training Accuracy : 0.2706
Epoch: 11	Batch: 100	Avg-Loss: 2.1120	Training Accuracy : 0.2677
Epoch: 11	Batch: 150	Avg-Loss: 2.1366	Training Accuracy : 0.2581
Epoch: 11	Batch: 200	Avg-Loss: 2.1323	Training Accuracy : 0.2700
Epoch: 11	Batch: 250	Avg-Loss: 2.1518	Training Accuracy : 0.2639
Epoch: 11	Batch: 300	Avg-Loss: 2.1316	Training Accuracy : 0.2611
Epoch: 11	Batch: 350	Avg-Loss: 2.1328	Training Accuracy : 0.2723
Epoch: 11	Batch: 400	Avg-Loss: 2.1386	Training Accuracy : 0.2722
Epoch: 11	Batch: 450	Avg-Loss: 2.1472	Training Accuracy : 0.2630
Epoch: 11	Batch: 500	Avg-Loss: 2.1445	Training Accuracy : 0.2634
Epoch: 11	Batch: 550	Avg-Loss: 2.1329	Training Accuracy : 0.2692
Epoch: 11	Batch: 600	Avg-Loss: 2.1495	Training Accuracy : 0.2575
Epoch: 11	Batch: 650	Avg-Loss: 2.1438	Training Accuracy : 0.2622
Epoch: 11, Validation Loss: 2.380, Validation Accuracy: 0.238
Epoch: 12	Batch: 50	Avg-Loss: 2.1070	Training Accuracy : 0.2709
Epoch: 12	Batch: 100	Avg-Loss:

In [19]:
from tqdm import tqdm

In [20]:
network = ClassificationNetwork(in_features, embedding, num_classes)
network.load_state_dict(torch.load("/content/gdrive/MyDrive/regnet_ours_data4/best_model_16.pt"))
network = network.to(device)

In [21]:
network.eval()
num_correct = 0
for x, y in tqdm(test_loader):
  x, y = x.to(device), y.to(device)
  outputs = network(x)
  num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

test_acc = num_correct / len(test_set)
print(test_acc)

100%|██████████| 141/141 [07:37<00:00,  3.24s/it]

0.22967626901431995


In [22]:
print(test_acc)

0.22967626901431995
